In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import cv2
from PIL import Image 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import sklearn as sk
from sklearn.model_selection import train_test_split
import pickle

print(tf.__version__)

2024-12-11 17:58:08.644934: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-11 17:58:08.651970: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 17:58:08.660111: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 17:58:08.662432: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 17:58:08.669246: I tensorflow/core/platform/cpu_feature_guar

2.17.0


In [2]:
gpus = tf.config.list_physical_devices('GPU')
gpuid = 0 #int(args.gpu_id)                                                                                                                           
if gpus:
  # Restrict TensorFlow to only allocate X GB of memory on the first GPU                                                                              
  try:
    tf.config.set_visible_devices(gpus[gpuid], 'GPU')
    tf.config.experimental.set_memory_growth(gpus[gpuid], True)
    '''
    tf.config.set_logical_device_configuration(
        gpus[gpuid],
        [tf.config.LogicalDeviceConfiguration(memory_limit=12000)])
    '''
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized                                                                                   
    print(e)

1 Physical GPUs, 1 Logical GPUs


I0000 00:00:1733965093.547278  583271 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733965093.569980  583271 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733965093.572196  583271 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733965093.575352  583271 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [3]:
with tf.device('/job:localhost/replica:0/task:0/device:CPU:0'):
    inputs = np.load("all_inputs_height.npy")

In [4]:
batchsize = 32
with tf.device('/job:localhost/replica:0/task:0/device:CPU:0'):
    targets = np.load("all_targets_height.npy")
    #X_train, X_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.2)

In [8]:
inputs.shape

(3467, 512, 512, 1)

In [10]:
inputs

array([[[[0.65385175],
         [0.6585279 ],
         [0.67108393],
         ...,
         [0.63729916],
         [0.64624695],
         [0.6634144 ]],

        [[0.66166691],
         [0.65849554],
         [0.66752423],
         ...,
         [0.64640875],
         [0.64741194],
         [0.64440238]],

        [[0.70396259],
         [0.69482064],
         [0.69257156],
         ...,
         [0.64896526],
         [0.63597236],
         [0.62385321]],

        ...,

        [[0.42276265],
         [0.42912156],
         [0.43277834],
         ...,
         [0.5774477 ],
         [0.56987525],
         [0.57178454]],

        [[0.41865282],
         [0.42881414],
         [0.43651603],
         ...,
         [0.57898484],
         [0.57625034],
         [0.57021504]],

        [[0.41687297],
         [0.42844199],
         [0.436872  ],
         ...,
         [0.57023122],
         [0.58531139],
         [0.58031164]]],


       [[[0.44314455],
         [0.44454773],
         [0.46

In [9]:
len(inputs[0][0].flatten())

512

Don't run any of the three below cells when using the alternate gridsearch method

In [13]:
batchsize = 32

with tf.device("/CPU:0") :
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
with tf.device("/GPU:0") :
    train_dataset = train_dataset.shuffle(buffer_size=200).batch(batchsize)
    val_dataset = val_dataset.shuffle(buffer_size=200).batch(batchsize)

In [ ]:
batchsize = 32

with tf.device('/job:localhost/replica:0/task:0/device:CPU:0'):
    targets = np.load("all_targets_with_height.npy")
    # train test split
    X_trainval, X_test, y_trainval, y_test = train_test_split(inputs, targets, test_size=0.2)
    X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2)

with tf.device("/CPU:0") :
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
with tf.device("/GPU:0") :
    train_dataset = train_dataset.shuffle(buffer_size=200).batch(batchsize)
    val_dataset = val_dataset.shuffle(buffer_size=200).batch(batchsize)

In [5]:
# updated batch processing
#X_trainval, X_test, y_trainval, y_test = train_test_split(inputs, targets, test_size=0.2)
with tf.device("/CPU:0") :
    X_train, X_val, y_train, y_val = train_test_split(inputs, targets, test_size=0.2)
    

# Batch processing\
with tf.device("/CPU:0") :
    batchsize = 32
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(200).batch(batchsize)
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batchsize)

In [7]:
targets.shape

(3470,)

In [18]:
for train_index, test_index in kf.split(inputs, targets):
    print(test_index)

[   3    5    7    9   16   23   24   28   31   37   41   43   45   48
   54   57   60   61   71   86   89   90   92   94  111  115  120  121
  123  124  127  131  139  140  141  146  152  163  175  178  179  180
  181  188  192  197  201  204  205  218  234  235  236  240  241  245
  249  260  261  277  279  282  284  289  291  300  302  324  328  334
  338  349  354  356  359  363  371  392  403  407  408  417  418  421
  426  429  434  436  439  446  450  453  456  458  467  489  499  506
  510  512  517  524  529  530  539  541  542  545  554  558  562  565
  574  587  590  591  592  598  599  622  625  626  627  628  632  638
  641  646  651  658  662  663  666  668  669  684  698  703  706  707
  713  715  724  728  729  730  736  738  750  751  754  755  757  759
  760  763  767  782  784  787  788  794  797  799  801  802  804  807
  812  813  815  816  818  823  824  825  830  837  839  843  851  853
  854  856  859  862  863  865  876  877  879  898  906  907  911  912
  913 

### Other gridsearch methods

In [5]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, AveragePooling2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
#from sklearn.model_selection import GridSearch

# Configure cross-validation and early stopping
n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Define hyperparameter grid
param_grid = {
    'k_dim': [3,9],
    'conv_dim': [16,32],
    'pool_dim': [3, 5],
    'pooling_type': ['max', 'average'],  
    'learning_rate': [1e-4, 1e-5],
}

# Perform cross-validation with early stopping
best_params = None
best_score = 0
test_scores = {}
i = 1

for k_dim in param_grid['k_dim']:
    for conv_dim in param_grid['conv_dim']:
        for learning_rate in param_grid['learning_rate']:
            for pooling_type in param_grid['pooling_type']:
                for pool_dim in param_grid['pool_dim']:
                    total_models = len(param_grid['pooling_type'])*len(param_grid['pool_dim'])*len(param_grid['k_dim'])*len(param_grid['conv_dim'])*len(param_grid['learning_rate'])
                    print(f"testing model {i} out of {total_models}")
                    print(f'testing params: {pooling_type}, {pool_dim}, {k_dim}, {conv_dim},{learning_rate}')
                    i += 1

                    num_folds = 1
                    best_rounds = []
                    for train_index, test_index in kf.split(inputs, targets):
                        with tf.device("/CPU:0") :
                            X_train_fold, X_test_fold = inputs[train_index], inputs[test_index]
                            y_train_fold, y_test_fold = targets[train_index], targets[test_index]

                        with tf.device("/CPU:0") :
                            #X_train, X_test, y_train, y_test = train_test_split(train_index, test_index, test_size=0.2)
                            X_train_fold, X_val, y_train_fold, y_val = train_test_split(X_train_fold, y_train_fold, test_size=0.2, random_state=42)


                        with tf.device("/CPU:0") :
                            batchsize = 32
                            train_dataset = tf.data.Dataset.from_tensor_slices((X_train_fold, y_train_fold)).shuffle(200).batch(batchsize)
                            val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batchsize)


                        early_stopping = EarlyStopping(
                        monitor="val_accuracy", 
                        patience=10,  # Stop if no improvement for 5 epochs
                        mode="max", 
                        restore_best_weights=True  # Automatically restores the best model weights
                        )

                        # Prepare the model
                        #k_dim=3
                        #conv_dim=16
                        #learning_rate=1e-4 
                        drop_rate=0
                        ff_dim=32
                        
                        model = Sequential([
                            Conv2D(conv_dim, kernel_size=(k_dim, k_dim), activation='relu', input_shape=(512, 512, 1)),
                            BatchNormalization(),
                            ReLU(),
                            MaxPooling2D(pool_size=(pool_dim, pool_dim)) if pooling_type == 'max' else AveragePooling2D(pool_size=(pool_dim, pool_dim)),
                            Dropout(drop_rate),
                            Conv2D(conv_dim, kernel_size=(k_dim, k_dim), padding='valid'),
                            BatchNormalization(),
                            ReLU(),
                            MaxPooling2D(pool_size=(pool_dim, pool_dim)) if pooling_type == 'max' else AveragePooling2D(pool_size=(pool_dim, pool_dim)),
                            Dropout(drop_rate),
                            Conv2D(conv_dim, kernel_size=(k_dim, k_dim), padding='valid'),
                            BatchNormalization(),
                            ReLU(),
                            MaxPooling2D(pool_size=(pool_dim, pool_dim)) if pooling_type == 'max' else AveragePooling2D(pool_size=(pool_dim, pool_dim)),
                            Dropout(drop_rate),
                            Flatten(),
                            Dense(ff_dim, activation='relu'),
                            Dropout(drop_rate),
                            Dense(1, activation='sigmoid')
                        ])
                        model.compile(optimizer=Adam(learning_rate=learning_rate),
                                    loss='binary_crossentropy',
                                    metrics=['accuracy'])

                        # Fit model on train fold and use validation for early stopping
                        model.fit(X_train_fold, y_train_fold, epochs = 270, validation_data=val_dataset,callbacks=[early_stopping])

                        # Predict on test set
                        #y_pred_test = model.predict(X_test_fold)
                        test_loss, test_acc = model.evaluate(X_test_fold, y_test_fold)
                        test_scores[f'fold:{num_folds}, pooling_type: {pooling_type}, pool_dim: {pool_dim}, k_dim: {k_dim}, conv_dim: {conv_dim}, learning_rate: {learning_rate}'] = {"loss":test_loss, "accuracy":test_acc}
                        
                        num_folds += 1

        # Compute average score across all folds
        #average_score = np.mean(test_scores)
        
        #if average_score > best_score:
            #best_score = average_score
            #best_params = {'learning_rate': learning_rate, 'pool_dim': pool_dim}

#print(f"Best Parameters: {best_params}")
#print(f"Best CV Average Accuracy: {best_score}")

testing model 1 out of 12
testing params: max, 3, 9, 32,0.001


/home/user/anaconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/270


I0000 00:00:1733965115.074833  583425 service.cc:146] XLA service 0x770cf8004710 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733965115.074857  583425 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4080 SUPER, Compute Capability 8.9
2024-12-11 17:58:35.114828: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-11 17:58:35.242046: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 3/70 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - accuracy: 0.5035 - loss: 2.3983

...

Epoch 1/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 60ms/step - accuracy: 0.4578 - loss: 0.8477 - val_accuracy: 0.4793 - val_loss: 0.7079
Epoch 2/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.5863 - loss: 0.6757 - val_accuracy: 0.4793 - val_loss: 0.6999
Epoch 3/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.6335 - loss: 0.6438 - val_accuracy: 0.5261 - val_loss: 0.6890
Epoch 4/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.6330 - loss: 0.6217 - val_accuracy: 0.5207 - val_loss: 0.7016
Epoch 5/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.6504 - loss: 0.6078 - val_accuracy: 0.5207 - val_loss: 0.7236
Epoch 6/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.6902 - loss: 0.5845 - val_accuracy: 0.5207 - val_loss: 0.7685
Epoch 7/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.6786 - loss: 0.5860 - val_accuracy: 0.5207 - val_loss: 0.8182
Epoch 8/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.6997 - loss: 0.5659 - val_accuracy: 0.

/home/user/anaconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 70ms/step - accuracy: 0.5362 - loss: 0.7161 - val_accuracy: 0.4793 - val_loss: 0.7027
Epoch 2/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - accuracy: 0.5776 - loss: 0.6799 - val_accuracy: 0.4865 - val_loss: 0.6982
Epoch 3/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step - accuracy: 0.6174 - loss: 0.6529 - val_accuracy: 0.5135 - val_loss: 0.6940
Epoch 4/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - accuracy: 0.6698 - loss: 0.6292 - val_accuracy: 0.5189 - val_loss: 0.7084
Epoch 5/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - accuracy: 0.6633 - loss: 0.6179 - val_accuracy: 0.5261 - val_loss: 0.7334
Epoch 6/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - accuracy: 0.6845 - loss: 0.5950 - val_accuracy: 0.5261 - val_loss: 0.7119
Epoch 7/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - accuracy: 0.6899 - loss: 0.5879 - val_accuracy: 0.5207 - val_loss: 0.7915
Epoch 8/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - accuracy: 0.7057 - loss: 0.5755 - val_accuracy: 0.

/home/user/anaconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - accuracy: 0.4740 - loss: 0.7099 - val_accuracy: 0.5207 - val_loss: 0.6922
Epoch 2/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.5485 - loss: 0.6829 - val_accuracy: 0.5207 - val_loss: 0.6935
Epoch 3/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.5713 - loss: 0.6739 - val_accuracy: 0.5207 - val_loss: 0.6964
Epoch 4/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.5985 - loss: 0.6651 - val_accuracy: 0.5207 - val_loss: 0.7009
Epoch 5/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.6109 - loss: 0.6551 - val_accuracy: 0.5207 - val_loss: 0.7029
Epoch 6/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.6484 - loss: 0.6434 - val_accuracy: 0.5207 - val_loss: 0.7070
Epoch 7/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.6421 - loss: 0.6447 - val_accuracy: 0.5207 - val_loss: 0.7070
Epoch 8/270
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.6357 - loss: 0.6436 - val_accuracy: 0.

In [6]:
test_scores

{'fold:1, pooling_type: max, pool_dim: 3, k_dim: 9, conv_dim: 32, learning_rate: 0.001': {'loss': 0.7603837251663208,
  'accuracy': 0.5504322648048401},
 'fold:2, pooling_type: max, pool_dim: 3, k_dim: 9, conv_dim: 32, learning_rate: 0.001': {'loss': 0.9335079789161682,
  'accuracy': 0.64985591173172},
 'fold:3, pooling_type: max, pool_dim: 3, k_dim: 9, conv_dim: 32, learning_rate: 0.001': {'loss': 0.6924248933792114,
  'accuracy': 0.5411255359649658},
 'fold:4, pooling_type: max, pool_dim: 3, k_dim: 9, conv_dim: 32, learning_rate: 0.001': {'loss': 0.7347667217254639,
  'accuracy': 0.6233766078948975},
 'fold:5, pooling_type: max, pool_dim: 3, k_dim: 9, conv_dim: 32, learning_rate: 0.001': {'loss': 0.7261780500411987,
  'accuracy': 0.5440115332603455},
 'fold:1, pooling_type: max, pool_dim: 5, k_dim: 9, conv_dim: 32, learning_rate: 0.001': {'loss': 0.6921393871307373,
  'accuracy': 0.5547550320625305},
 'fold:2, pooling_type: max, pool_dim: 5, k_dim: 9, conv_dim: 32, learning_rate: 0.0

In [13]:
def make_results_df(test_scores):
    # Initialize a dictionary to aggregate results
    aggregated_results = {}

    for key, metrics in test_scores.items():
        # Parse the key to extract the hyperparameter combination (remove fold info)
        params = key
        
        # Initialize an entry in the results dictionary if not already present
        if params not in aggregated_results:
            aggregated_results[params] = {'loss': [], 'accuracy': []}
        
        # Append loss and accuracy for this fold
        aggregated_results[params]['loss'].append(metrics['loss'])
        aggregated_results[params]['accuracy'].append(metrics['accuracy'])

    # Create a DataFrame to store averaged results
    results_list = []
    for params, metrics in aggregated_results.items():
        loss = metrics['loss'][0]
        accuracy = metrics['accuracy'][0]
        results_list.append({'params': params, 'loss': loss, 'accuracy': accuracy})

    # Convert to DataFrame
    results_df = pd.DataFrame(results_list).set_index('params')
    return results_df

raw_result_df = make_results_df(test_scores) 
raw_result_df.head()

,loss,accuracy
params,,
"fold:1, pooling_type: max, pool_dim: 3, k_dim: 3, conv_dim: 16, learning_rate: 0.001",0.665986,0.629683
"fold:2, pooling_type: max, pool_dim: 3, k_dim: 3, conv_dim: 16, learning_rate: 0.001",0.692688,0.541787
"fold:3, pooling_type: max, pool_dim: 3, k_dim: 3, conv_dim: 16, learning_rate: 0.001",0.822476,0.624820
"fold:4, pooling_type: max, pool_dim: 3, k_dim: 3, conv_dim: 16, learning_rate: 0.001",0.692710,0.541126
"fold:5, pooling_type: max, pool_dim: 3, k_dim: 3, conv_dim: 16, learning_rate: 0.001",0.692670,0.541126


In [14]:
raw_result_df.to_csv("2024-12-03_gridsearch_raw_results_height.csv")

In [7]:
def calculate_average_scores_df(test_scores):
    # Initialize a dictionary to aggregate results
    aggregated_results = {}

    for key, metrics in test_scores.items():
        # Parse the key to extract the hyperparameter combination (remove fold info)
        params = ', '.join([part for part in key.split(', ') if not part.startswith('fold')])
        
        # Initialize an entry in the results dictionary if not already present
        if params not in aggregated_results:
            aggregated_results[params] = {'loss': [], 'accuracy': []}
        
        # Append loss and accuracy for this fold
        aggregated_results[params]['loss'].append(metrics['loss'])
        aggregated_results[params]['accuracy'].append(metrics['accuracy'])

    # Create a DataFrame to store averaged results
    results_list = []
    for params, metrics in aggregated_results.items():
        avg_loss = sum(metrics['loss']) / len(metrics['loss'])
        avg_accuracy = sum(metrics['accuracy']) / len(metrics['accuracy'])
        results_list.append({'params': params, 'average_loss': avg_loss, 'average_accuracy': avg_accuracy})

    # Convert to DataFrame
    results_df = pd.DataFrame(results_list).set_index('params')
    return results_df

In [8]:
averaged_results = calculate_average_scores_df(test_scores)
averaged_results

,average_loss,average_accuracy
params,,
"pooling_type: max, pool_dim: 3, k_dim: 9, conv_dim: 32, learning_rate: 0.001",0.769452,0.581760
"pooling_type: max, pool_dim: 5, k_dim: 9, conv_dim: 32, learning_rate: 0.001",0.730809,0.593307
"pooling_type: average, pool_dim: 3, k_dim: 9, conv_dim: 32, learning_rate: 0.001",0.752754,0.551185
"pooling_type: average, pool_dim: 5, k_dim: 9, conv_dim: 32, learning_rate: 0.001",0.693488,0.561271
"pooling_type: max, pool_dim: 3, k_dim: 9, conv_dim: 32, learning_rate: 0.0001",0.714853,0.689331
"pooling_type: max, pool_dim: 5, k_dim: 9, conv_dim: 32, learning_rate: 0.0001",0.683571,0.650684
"pooling_type: average, pool_dim: 3, k_dim: 9, conv_dim: 32, learning_rate: 0.0001",0.756785,0.632224
"pooling_type: average, pool_dim: 5, k_dim: 9, conv_dim: 32, learning_rate: 0.0001",0.652729,0.634526
"pooling_type: max, pool_dim: 3, k_dim: 9, conv_dim: 32, learning_rate: 1e-05",0.697981,0.618104


In [17]:
averaged_results.to_csv("2024-12-03_gridsearch_averaged_results_height.csv")

In [14]:
def find_best_hyperparameters_df(results_df):
    # Find the row with the highest average_accuracy
    best_row = results_df.loc[results_df['average_accuracy'].idxmax()]
    
    # Extract the best parameters, loss, and accuracy
    best_params = best_row.name  # The index contains the parameter combination
    best_loss = best_row['average_loss']
    best_accuracy = best_row['average_accuracy']
    
    return best_params, best_loss, best_accuracy

best_params, best_loss, best_accuracy = find_best_hyperparameters_df(averaged_results)
print(f"Best Parameters: {best_params}")
print(f"Best Loss: {best_loss:.4f}")
print(f"Best Accuracy: {best_accuracy:.4f}")

Best Parameters: pooling_type: average, pool_dim: 5, k_dim: 3, conv_dim: 32, learning_rate: 0.0001
Best Loss: 0.5036
Best Accuracy: 0.7335


In [7]:
def find_best_hyperparameters(averaged_results):
    # Initialize variables to track the best hyperparameters
    best_params = None
    best_accuracy = 0
    best_loss = None

    # Iterate over the averaged results
    for params, metrics in averaged_results.items():
        if metrics['average_accuracy'] > best_accuracy:
            best_accuracy = metrics['average_accuracy']
            best_loss = metrics['average_loss']
            best_params = params

    return best_params, best_loss, best_accuracy

In [19]:
best_params, best_loss, best_accuracy = find_best_hyperparameters(averaged_results)
print(f"Best Parameters: {best_params}")
print(f"Best Loss: {best_loss:.4f}")
print(f"Best Accuracy: {best_accuracy:.4f}")

Best Parameters: pooling_type: max, pool_dim: 5
Best Loss: 0.6461
Best Accuracy: 0.5852


In [ ]:
y_test

### GridSearchCV


In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, AveragePooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping


# Define the model function
def build_model(pool_dim):
    k_dim=3
    conv_dim=16
    learning_rate=1e-4 
    drop_rate=0
    ff_dim=32
    
    model = Sequential([
        Conv2D(conv_dim, kernel_size=(k_dim, k_dim), activation='relu', input_shape=(512, 512, 2)),
        BatchNormalization(),
        ReLU(),
        AveragePooling2D(pool_size=(pool_dim, pool_dim)),
        Dropout(drop_rate),
        Conv2D(conv_dim, kernel_size=(k_dim, k_dim), padding='valid'),
        BatchNormalization(),
        ReLU(),
        AveragePooling2D(pool_size=(pool_dim, pool_dim)),
        Dropout(drop_rate),
        Conv2D(conv_dim, kernel_size=(k_dim, k_dim), padding='valid'),
        BatchNormalization(),
        ReLU(),
        AveragePooling2D(pool_size=(pool_dim, pool_dim)),
        Dropout(drop_rate),
        Flatten(),
        Dense(ff_dim, activation='relu'),
        Dropout(drop_rate),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Define parameter grid
param_grid = {
   # 'k_dim': [3, 9, 15, 27],
   # 'conv_dim': [16, 32, 48],
    'pool_dim': [3, 5],
   # 'learning_rate': [1e-2, 1e-3, 1e-4, 1e-5],
    'epochs': [50]
}

model_checkpoint = ModelCheckpoint(
  filepath='checkpoint.model3.keras',
  monitor='val_loss',
  mode='min',
  save_best_only=True)

early_stopping = EarlyStopping(
    monitor="val_accuracy", 
    patience=5,  # Stop if no improvement for 5 epochs
    mode="max", 
    restore_best_weights=True  # Automatically restores the best model weights
)

# Wrap the model
model = KerasClassifier(build_fn=build_model, batch_size=batchsize, pool_dim = None, verbose=1)

# Perform GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=2,verbose=3, n_jobs=1)


In [ ]:
grid_result = grid.fit(X_train, 
                        y_train, 
                        validation_data = val_dataset,
                        callbacks=[early_stopping]
                        )
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
print(model.get_params().keys())

In [ ]:
print(grid.get_params().keys())

In [94]:
param_grid = {
    'k_dim': [3, 9, 15, 27],
    'conv_dim': [16],
    'pool_dim': [3, 5],
    'learning_rate': [1e-2,1e-3,1e-4,1e-5]
}

In [95]:
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

# Wrap the model
model = KerasClassifier(build_fn=build_model, verbose=0)

# Perform GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=2, n_jobs=-1)

In [ ]:
grid.get_params().keys()

In [ ]:
grid_results = grid.fit(inputs,targets,
                  #validation_data=val_dataset, 
                  epochs=50)
                  #callbacks=[model_checkpoint])

In [ ]:
conv_dim = 16
ff_dim = 32
k_dim = 9
pool_dim = 5
drop_rate = 0.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(conv_dim, kernel_size = (k_dim, k_dim), activation='relu', input_shape=(512, 512, 2)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.ReLU(),
  tf.keras.layers.AveragePooling2D(pool_dim,pool_dim),
  tf.keras.layers.Dropout(drop_rate),
  tf.keras.layers.Conv2D(conv_dim, kernel_size = (k_dim, k_dim), padding ='valid'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.ReLU(),
  tf.keras.layers.AveragePooling2D(pool_dim,pool_dim),
  tf.keras.layers.Dropout(drop_rate),
  tf.keras.layers.Conv2D(conv_dim, kernel_size = (k_dim, k_dim), padding ='valid'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.ReLU(),
  tf.keras.layers.AveragePooling2D(pool_dim,pool_dim),
  tf.keras.layers.Dropout(drop_rate),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(ff_dim, activation='relu'),
  tf.keras.layers.Dropout(drop_rate),
  tf.keras.layers.Dense(1, activation='sigmoid')  
])

model.summary()

In [26]:
#model.compile(optimizer=Adam(learning_rate=1e-4),
#model.compile(optimizer=Adam(learning_rate=1e-2),
#model.compile(optimizer=Adam(learning_rate=1e-5),
model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='binary_crossentropy',  
              metrics=['accuracy'])

In [ ]:
model_checkpoint = ModelCheckpoint(
  filepath='checkpoint.modelG6.keras',
  monitor='val_loss',
  mode='min',
  save_best_only=True)

optim = model.fit(train_dataset,
                  validation_data=val_dataset, 
                  epochs=50, 
                  callbacks=[model_checkpoint])

In [ ]:
optim.history.keys()

In [30]:
train_acc_arr = np.array(optim.history['accuracy'])
val_acc_arr = optim.history['val_accuracy']
val_loss_arr = optim.history['val_loss']

In [ ]:
epochs = []
for i in range(100):
    if train_acc_arr[i] > 0.98:
        epochs.append(i)

first_epoch_exceed_98 = epochs[0]
first_epoch_exceed_98

In [ ]:
plt.plot(optim.history['accuracy'])
plt.plot(optim.history['val_accuracy'])
plt.legend(['train', 'validation'], loc='lower right')
plt.title('Accuracy')
plt.show()

In [ ]:
plt.plot(optim.history['loss'])
plt.plot(optim.history['val_loss'])
plt.legend(['train', 'validation'], loc='upper left')
plt.title('Loss')
plt.show()

In [ ]:
def calculate_quarter_stats(val_loss, val_acc):
    x = np.arange(25)
    val_acc_diff = np.gradient(val_acc)
    val_loss_diff = np.gradient(val_loss)
    slope, _ = np.polyfit(x, val_loss, 1)
    return {
        "min_loss": np.min(val_loss),
        "max_loss": np.max(val_loss),
        "min_acc": np.min(val_acc),
        "max_acc": np.max(val_acc),
        "std_loss" : np.std(val_loss),
        "r_loss": np.corrcoef(x, val_loss)[0, 1],
        "slope_loss": slope,
        "avg_loss_gradient": np.mean(val_loss_diff),
        "avg_acc_gradient": np.mean(val_acc_diff),
    }

# Calculate statistics for each quarter
quarter_1 = calculate_quarter_stats(val_loss_arr[:25], val_acc_arr[:25])
quarter_2 = calculate_quarter_stats(val_loss_arr[25:50], val_acc_arr[25:50])
quarter_3 = calculate_quarter_stats(val_loss_arr[50:75], val_acc_arr[50:75])
quarter_4 = calculate_quarter_stats(val_loss_arr[75:], val_acc_arr[75:])

# Store results in a DataFrame
data = pd.DataFrame([quarter_1, quarter_2, quarter_3, quarter_4], 
                    index=["Q1", "Q2", "Q3", "Q4"])


print(data)

In [ ]:
data

In [36]:
val_acc_diff1 = np.gradient(val_acc_arr)
val_loss_diff1 = np.gradient(val_loss_arr)

In [ ]:
val_acc_diff1

In [ ]:
sum(val_acc_diff1 > 0)

In [ ]:
plt.plot(val_acc_diff1)
plt.axhline(y=0,color='r')
plt.title("Accuracy Gradient")
plt.ylabel("Differences in accuracy between epochs")
plt.ylim(-.2,.2)

In [ ]:
plt.plot(val_loss_diff1)
plt.axhline(y=0,color='r')
plt.title("Loss Gradient")
plt.ylabel("Differences in loss between epochs")
plt.ylim(-60,60)

In [33]:
x = np.arange(25)
# 1st quarter
val_loss1 = val_loss_arr[:25]
val_acc1 = val_acc_arr[:25]
quarter_1 = {
"min_loss": np.min(val_loss1),
"min_acc" : np.min(val_acc1),
"max_loss" : np.max(val_loss1),
"max_acc" : np.max(val_acc1),
"r_loss" : np.corrcoef(x,val_loss1)[0,1],
"r_squared_loss" : np.corrcoef(x,val_loss1)[0,1]**2
}

# 2nd quarter
val_loss2 = val_loss_arr[25:50]
val_acc2 = val_acc_arr[25:50]
quarter_2 = {
"min_loss" : np.min(val_loss2),
"min_acc" : np.min(val_acc2),
"max_loss" : np.max(val_loss2),
"max_acc" : np.max(val_acc2),
"r_loss" : np.corrcoef(x,val_loss2)[0,1],
"r_squared_loss" : np.corrcoef(x,val_loss2)[0,1]**2
}

# 3rd quarter
val_loss3 = val_loss_arr[50:75]
val_acc3 = val_acc_arr[50:75]
quarter_3 = {
"min_loss":np.min(val_loss3),
"min_acc":np.min(val_acc3),
"max_loss" : np.max(val_loss3),
"max_acc" : np.max(val_acc3),
"r_loss" : np.corrcoef(x,val_loss3)[0,1],
"r_squared_loss" : np.corrcoef(x,val_loss3)[0,1]**2
}

# 4th quarter
val_loss4 = val_loss_arr[75:]
val_acc4 = val_acc_arr[75:]
quarter_4 = {
"min_loss" : np.min(val_loss4),
"min_acc" : np.min(val_acc4),
"max_loss" : np.max(val_loss4),
"max_acc" : np.max(val_acc4),
"r_loss" : np.corrcoef(x,val_loss4)[0,1],
"r_squared_loss" : np.corrcoef(x,val_loss4)[0,1]**2
}

In [ ]:
quarter_3

In [75]:

x = np.arange(50)
r = np.corrcoef(x,optim.history['val_loss'])[0, 1]

In [ ]:
r**2

In [ ]:
len(val_loss3)

In [ ]:
np.std(val_acc_arr[2:24])

In [ ]:
np.std(val_loss_arr[2:24])

In [ ]:
np.array(val_loss_arr[2:24]).min()

In [ ]:
np.array(optim.history['val_accuracy']).max()

In [37]:
x = np.arange(50)
r = np.corrcoef(x,optim.history['val_loss'])[0, 1]

In [49]:
val_acc_diff1 = np.gradient(val_acc_arr)
val_loss_diff1 = np.gradient(val_loss_arr)

val_acc_diff2 = np.gradient(val_acc_diff1)
val_loss_diff2 = np.gradient(val_loss_diff1)

In [ ]:
plt.plot(val_acc_diff1)
plt.axhline(y=0,color='r')
#plt.plot(val_acc_diff2)

In [ ]:
plt.plot(val_loss_diff1)
plt.axhline(y=0,color='r')
plt.ylim(-3,3)

In [ ]:
# maybe the rate of change needs to roughly 

In [ ]:
# want to measure the rate of change 
plt.hist(val_loss_diff1)

In [39]:
from tensorflow.keras.models import load_model

best_model = load_model('checkpoint.modelG6.keras')

In [ ]:
test_loss, test_acc = best_model.evaluate(X_test, y_test)

In [ ]:
'Test loss:{0}, Test accuracy: {1}'.format(test_loss,test_acc)

In [ ]:
# Predict the values from the testing dataset
Y_pred_probs = best_model.predict(X_test)
Y_pred = np.array(Y_pred_probs>=.5)
# compute the confusion matrix
#confusion_mtx = tf.math.confusion_matrix(Y_true, Y_pred) 

In [ ]:
plt.hist(Y_pred_probs.flatten())
plt.title('Probability distribution of Predictions')
plt.xlabel('Probability')
plt.ylabel('freq')

In [ ]:
plt.hist(y_test)
plt.title('Probability distribution of True values')
plt.xlabel('Probability')
plt.ylabel('freq')

In [ ]:
sum(y_test)

In [44]:
Y_pred = Y_pred.astype(float).flatten()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, Y_pred) 
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()


In [ ]:
cm

In [17]:
TP=152
TN=175
FP=13
FN=1

In [ ]:
recall = TP/(TP+FN)
precision = TP/(TP+FP)

print(recall,precision)

In [ ]:
(TP+TN)/(TP+TN+FP+FN)